Przed oddaniem zadania upewnij się, że wszystko działa poprawnie.
**Uruchom ponownie kernel** (z paska menu: Kernel$\rightarrow$Restart) a następnie
**wykonaj wszystkie komórki** (z paska menu: Cell$\rightarrow$Run All).

Upewnij się, że wypełniłeś wszystkie pola `TU WPISZ KOD` lub `TU WPISZ ODPOWIEDŹ`, oraz
że podałeś swoje imię i nazwisko poniżej:

In [ ]:
NAME = ""

---

# 1. Grafowe sieci neuronowe

W niniejszym zeszycie opiszemy i przeprowadzimy ewaluację 3 najpopularniejszych architektur grafowych sieci neuronowych, mianowicie:
- Graph Convolutional Network (**GCN**)
- Graph Sample and Aggregate (**GraphSAGE**)
- Graph Attention Network (**GAT**)

Wykorzystamy poznany w ramach poprzedniego laboratorium zbiór Cora i sprawdzimy jakość działania powyższych modeli w zadaniu klasyfikacji węzłów. Implementacje modeli są dostępne w bibliotece PyTorch-Geometric, natomiast do uczenia modeli wykorzystamy bibliotekę PyTorch-Lightning.

## 1.1. Załadowanie zbioru
Zbiory danych w PyTorch-Lightningu są przekazywane jako instancje obiektów `DataLoader` do metody `fit()` klasy `Trainer`. Można jednak użyć klasy `LightningDataModule` i w niej zdefiniować wszystkie data loadery wraz z wczytaniem właściwego zbioru. Zauważmy, że dotychczas rozważany zbiór danych Cora nie jest podzielony na mniejsze części (tj. mamy dokładnie jeden obiekt `Data`, który opisuje cały graf). Można wykorzystywać metody próbkowania grafów (np. `NeighborSampler`) i przez to uzyskać podział na mini-paczki, jednak zbiór Cora jest na tyle mały, że nie opłaca się go bardziej dzielić. Wykorzystamy natomiast fakt, że wczytywany obiekt `Planetoid` jest iterowalny i zwiera dokładnie jeden element. Możemy go zatem opakować w obiekt `DataListLoader` (dedykowana wersja `DataLoader`a zaimplementowana w PyTorch-Geometricu).

In [ ]:
from IPython.display import Code, display

display(Code("src/dataset.py"))

In [ ]:
from src.dataset import GraphData


datamodule = GraphData(dataset_name="Cora")

## 1.2. Trenowanie modeli
Wszystkie wymienione modele grafowych sieci neuronowych zostały zaproponowane jako warstwy (z perspektywy głębokiego uczenia maszynowego), które możemy połączyć z dowolnymi innymi warstwami, aby utworzyć model (tzn. możemy wykorzystać znane aktywacje, warstwy liniowe itd.). Istotne dla nas jest jednak to, że grafowe sieci neuronowe (ang. *Graph Neural Networks - GNN*) nie są związane z jakąkolwiek funkcją kosztu i możemy je uczyć zarówno w scenariuszu nadzorowanym jak i nienadzorowanym. Temat modeli nienadzorowanych jest bardziej złożony, zatem na razie go pominiemy i wrócimy do niego w następnym zeszycie.

Na chwilę obecną będziemy rozważać scenariusz nadzorowanej klasyfikacji wierzchołków. Przeanalizujmy implementację klasy `SupervisedNodeClassificationGNN`:

In [ ]:
display(Code("src/supervised.py"))

Jak już wspomnieliśmy modele grafowych sieci neuronowych mogą być stosowane w scenariuszu induktywnym. Pomimo, że przekazujemy pełną macierz atrybutów wierzchołków `data.x` oraz pełny zbiór krawędzi `data.edge_index`, funkcję kosztu oraz metrykę AUC obliczamy na wybranym pozbiorze wierzchołków. Podzbiór ten jest określany przez maskę zbioru treningowego `data.train_mask`, walidacyjnego `data.val_mask` oraz testowego `data.test_mask`. Na cele tego laboratorium możemy uznać taki scenariusz jako induktywny, jednak w rzeczywistych zastosowaniach musielibyśmy zadbać oto, aby na danym etapie uczenia (trening, walidacja, testowanie) pozostawiać tylko odpowiedni podzbiór krawędzi i wierzchołków (więcej o tym na kolejnym wykładzie!).


In [ ]:
from src.supervised import SupervisedNodeClassificationGNN

Dla wszystkich modeli zdefiniujmy sobie zbiór wspólnych hiperparametrów:

In [ ]:
hparams = {
    "num_epochs": 10,
    "hidden_dim":  256,
    "emb_dim": 128,
}

# 1.3. Graph Convolutional Network (GCN)
Grafowe sieci neuronowe po raz pierwszy w literaturze były proponowane już w okolicach 2008 roku, jednak dopiero rozwój i popularyzacja uczenia głębokiego pozwoliła na efektywne implementacje. Najpopularniejszym obecnie modelem grafowej sieci neuronowej jest **grafowa konwolucja** (GCN - *Graph Convolutional Network*), która została zaproponowana przez Kipfa w 2016 roku – [artykuł](https://arxiv.org/pdf/1609.02907.pdf). Praca ma już niespełna 23 tysiące cytowań i wiele obecnych GNNów jest oparta na niej.

Model GCN w każdej warstwie oblicza nowe cechy wierzchołków $H^{(l+1)}$ na podstawie obecnych cech $H^{(l)}$ w następujący sposób:

$$H^{(l+1)} = \hat{D}^{-\frac{1}{2}}\hat{A}\hat{D}^{-\frac{1}{2}}H^{(l)}W^{(l)},$$
gdzie:
- $\hat{A} = A + I$ to macierz sąsiedztwa grafu z dołączonymi pętlami zwrotnymi na każdym wierzchołku (krawędź z danego wierzchołka do samego siebie)
- $\hat{D}$ to macierz stopnii węzłów (macierz diagonalna)
- $\hat{D}^{-\frac{1}{2}}\hat{A}\hat{D}^{-\frac{1}{2}}$ to tzw. symetryczna normalizacja macierzy sąsiedztwa
- $W^{(l)}$ to macierz wyuczalnych parametrów

Powyższa funkcja odgrywa rolę wcześniej wprowadzonej funkcji $\phi$. Poprzez dodanie pętli na każdym wierzchołku, uśredniane są cechy zarówno sąsiadów jak i cechy danego wierzchołka. Natomiast symetryczna normalizacja pozwala uwzględnić stopień danego wierzchołka oraz stopień sąsiada.

Często definicja powyższej reguły *propagacji* uwzględnia również funkcję aktywacji. W celu uniknięcia pomyłek, tutaj ją pomijamy – PyTorch-Geometric też nie stosuje funkcji aktywacji w implementacjach poszczególnych warstw.

Zobaczmy jak zaimplementować grafową sieć neuronową wykorzystującą konwolucję grafową. Utworzymy sieć dwuwarstwową (uwzględniającą zatem sąsiedztwo dwu-skokowe) – zobacz klasa `GCNModel`. Następnie wykorzystamy klasę `SupervisedNodeClassificationGNN` i bibliotekę `PyTorch-Lightning`, aby nauczyć model klasyfikacji wierzchołków na zbiorze Cora.

In [ ]:
import matplotlib.pyplot as plt
from torch import nn
from torch_geometric.nn import GCNConv

from src.trainer import get_default_trainer
from src.utils import visualize_embeddings


class GCNModel(nn.Module):
    def __init__(self, in_dim: int, hidden_dim: int, out_dim: int):
        super().__init__()
        self.conv1 = GCNConv(in_dim, hidden_dim)
        self.act1 = nn.ReLU()
        self.conv2 = GCNConv(hidden_dim, out_dim)
        self.act2 = nn.ReLU()

    def forward(self, x, edge_index):
        z = self.act1(self.conv1(x, edge_index))
        z = self.act2(self.conv2(z, edge_index))
        return z


    
def evaluate_gcn():
    gnn = GCNModel(
        in_dim=datamodule.num_node_features,
        hidden_dim=hparams["hidden_dim"],
        out_dim=hparams["emb_dim"],
    )
    
    model=SupervisedNodeClassificationGNN(
        gnn=gnn, 
        emb_dim=hparams["emb_dim"],
        num_classes=datamodule.num_classes,
    )

    trainer = get_default_trainer(
        num_epochs=hparams["num_epochs"],
        model_name="supervised_GCN",
    )
    
    trainer.fit(model=model, datamodule=datamodule)
    
    test_auc = trainer.test(model=model, datamodule=datamodule, verbose=False)[0]["test/auc"]
    z, y = trainer.predict(model=model, datamodule=datamodule)[0]
    
    fig = visualize_embeddings(z=z, y=y)
    fig.suptitle(f"GCN - test AUC: {test_auc * 100.:.2f} [%]")
    
    plt.show()
    

evaluate_gcn()

## 1.4. Graph Sample and Aggregate (GraphSAGE)
W 2017 roku Hamilton opublikował [pracę](https://arxiv.org/pdf/1706.02216.pdf), w której rozważał induktywne uczenie grafowych sieci neuronowych oraz zaproponował sposób na osiągnięcie lepszej skalowalności metod GNNowych. Zaproponowana metoda (a właściwie rodzina metod) opiera się na idei próbkowania sąsiedztwa grafu (losowo wybrany podzbiór sąsiadów danego węzła) i następnie agregacji cech tak uzyskanej próbki sąsiadów. Zostały rozważone 3 metody agregacji: uśrednienie, LSTM oraz max pooling. Metoda była inspirowana bezpośrednio algorytmem Weisfeiler-Lehman test, a reguła propagacji jest zdefiniowana następująco (z perspektywy pojedynczego wierzchołka):

$$h^{(l)}_{\mathcal{N}(u)} = \text{AGGREGATE}^{(l)}(\{h^{(l)}_v, \forall v\in\mathcal{N}(u) \}) $$
$$h^{(l+1)}_u = W^{(l)} \cdot \text{CONCAT}(h^{(l)}_u, h^{(l)}_{\mathcal{N}(u)})$$

Widzimy zatem, że najpierw agregujemy cechy sąsiadów za pomocą wybranej metody agregacji, a następnie konkatenujemy wektor cech danego wierzchołka ze zagregowanym wektorem sąsiadów, po czym przemnażamy taki wektor przez macierz wag.

W PyTorch-Geometricu, metoda GraphSAGE (tutaj: `SAGEConv`) jest zaimplementowana delikatnie inaczej – reguła propagacji jest określona następująco (dla wariantu z uśrednieniem):

$$h^{(l+1)}_u = W^{(l)}_1 \cdot h^{(l)}_u + W^{(l)}_2 \cdot \text{mean}_{v\in\mathcal{N}(u)} h^{(l)}_v$$

Cechy danego wierzchołka i zagregowanego sąsiedztwa są przekształcane przez osobne wyuczalne macierze.

Analogicznie do modelu GCN przeprowadźmy ewaluację:

In [ ]:
from torch_geometric.nn import SAGEConv


class GraphSAGEModel(nn.Module):
    def __init__(self, in_dim: int, hidden_dim: int, out_dim: int):
        super().__init__()
        self.conv1 = SAGEConv(in_dim, hidden_dim)
        self.act1 = nn.ReLU()
        self.conv2 = SAGEConv(hidden_dim, out_dim)
        self.act2 = nn.ReLU()

    def forward(self, x, edge_index):
        z = self.act1(self.conv1(x, edge_index))
        z = self.act2(self.conv2(z, edge_index))
        return z
    
    
    
def evaluate_graphsage():
    gnn = GraphSAGEModel(
        in_dim=datamodule.num_node_features,
        hidden_dim=hparams["hidden_dim"],
        out_dim=hparams["emb_dim"],
    )
    
    model=SupervisedNodeClassificationGNN(
        gnn=gnn, 
        emb_dim=hparams["emb_dim"],
        num_classes=datamodule.num_classes,
    )

    trainer = get_default_trainer(
        num_epochs=hparams["num_epochs"],
        model_name="supervised_GraphSAGE",
    )
    
    trainer.fit(model=model, datamodule=datamodule)
    
    test_auc = trainer.test(model=model, datamodule=datamodule, verbose=False)[0]["test/auc"]
    z, y = trainer.predict(model=model, datamodule=datamodule)[0]
    
    fig = visualize_embeddings(z=z, y=y)
    fig.suptitle(f"GraphSAGE - test AUC: {test_auc * 100.:.2f} [%]")
    
    plt.show()
    

evaluate_graphsage()

## 1.5. Graph Attention Network (GAT)
Również w 2017 roku, Velickovic opublikował [pracę](https://arxiv.org/pdf/1710.10903.pdf), która przetłumaczyła mechanizm uwagi znany z przetwarzania języka naturalnego do dziedziny grafów. Powstała metoda nazywana Graph Attention (GAT). Reguła propagacji jest określona następująco:

$$h^{(l+1)}_u = \alpha_{u,u} W^{(l)}h^{(l)}_u + \sum_{v \in \mathcal{N}(u)} \alpha_{u,v}W^{(l)}h^{(l)}_v $$

$$\alpha_{i, j} = \frac{\exp(\text{LeakyReLU}(a^T[W^{(l)}h^{(l)}_i || W^{(l)}h^{(l)}_j]))}{\sum_{k\in i\cup \mathcal{N}(i)} \exp(\text{LeakyReLU}(a^T[W^{(l)}h^{(l)}_i || W^{(l)}h^{(l)}_k]))},$$

gdzie:
- $W^{(l)}$ to wyuczalna macierz parametrów
- $a$ to wyuczalne parametry mechanizmu uwagi (ang. *attention parameters*)
- $\alpha$ to współczynniki mechanizmu uwagi (ang. *attention coefficients*)

Metoda pozwala również na wykorzystanie wielu głowic uwagi (ang. *multi-headed attention*).

In [ ]:
from torch_geometric.nn import GATConv


class GATModel(nn.Module):
    def __init__(self, in_dim: int, hidden_dim: int, out_dim: int):
        super().__init__()
        self.conv1 = GATConv(in_dim, hidden_dim, heads=1)
        self.act1 = nn.ReLU()
        self.conv2 = GATConv(hidden_dim, out_dim, heads=1)
        self.act2 = nn.ReLU()

    def forward(self, x, edge_index):
        z = self.act1(self.conv1(x, edge_index))
        z = self.act2(self.conv2(z, edge_index))
        return z
    
    
    
def evaluate_gat():
    gnn = GATModel(
        in_dim=datamodule.num_node_features,
        hidden_dim=hparams["hidden_dim"],
        out_dim=hparams["emb_dim"],
    )
    
    model=SupervisedNodeClassificationGNN(
        gnn=gnn, 
        emb_dim=hparams["emb_dim"],
        num_classes=datamodule.num_classes,
    )

    trainer = get_default_trainer(
        num_epochs=hparams["num_epochs"],
        model_name="supervised_GAT",
    )
    
    trainer.fit(model=model, datamodule=datamodule)
    
    test_auc = trainer.test(model=model, datamodule=datamodule, verbose=False)[0]["test/auc"]
    z, y = trainer.predict(model=model, datamodule=datamodule)[0]
    
    fig = visualize_embeddings(z=z, y=y)
    fig.suptitle(f"GAT - test AUC: {test_auc * 100.:.2f} [%]")
    
    plt.show()
    

evaluate_gat()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./data/logs --port 6006

# Zadania

## 1.1. Opis i ewaluacja wybranej warstwy GNN (3 pkt)
Korzystając z listy zaimplementowanych w PyTorch-Geometricu warstw grafowych sieci neuronowych, wybierz jedną z nich, a następnie:

a) Sprawdź jak w tej warstwie realizowana jest reguła propagacji – porównaj wzór z wyżej badanymi modelami oraz zastanów się jak rozumiesz zasadę działania. **Na zajęciach, powinieneś potrafić to wyjaśnić własnymi słowami.**

b) Analogicznie do modeli GCN, GraphSAGE oraz GAT, zaimplementuj model dwuwarstwowy i przeprowadź jego ewaluację w zadaniu nadzorowanej klasyfikacji wierzchołków.

In [ ]:
class SelectedGNNModel(nn.Module):
    def __init__(self, in_dim: int, hidden_dim: int, out_dim: int):
        super().__init__()
        # TU WPISZ KOD
        raise NotImplementedError()

    def forward(self, x, edge_index):
        # TU WPISZ KOD
        raise NotImplementedError()
    
    
    
def evaluate_selected_gnn():
    gnn = SelectedGNNModel(
        in_dim=datamodule.num_node_features,
        hidden_dim=hparams["hidden_dim"],
        out_dim=hparams["emb_dim"],
    )
    
    model=SupervisedNodeClassificationGNN(
        gnn=gnn, 
        emb_dim=hparams["emb_dim"],
        num_classes=datamodule.num_classes,
    )

    trainer = get_default_trainer(
        num_epochs=hparams["num_epochs"],
        model_name="supervised_Selected",
    )
    
    trainer.fit(model=model, datamodule=datamodule)
    
    test_auc = trainer.test(model=model, datamodule=datamodule, verbose=False)[0]["test/auc"]
    z, y = trainer.predict(model=model, datamodule=datamodule)[0]
    
    fig = visualize_embeddings(z=z, y=y)
    fig.suptitle(f"Selected - test AUC: {test_auc * 100.:.2f} [%]")
    
    plt.show()
    

evaluate_selected_gnn()

## 1.2. Badanie głębokości grafowej sieci neuronowej (5 pkt)
a) Dla trzech powyżej przedstawionych modeli GNNowych oraz wybranej w zadaniu 1.1 warstwy, zaimplementuj klasę, która utworzy grafową sieci neuronową z dowolnie określoną liczbą warstw (w poprzednich przykładach używaliśmy 2-warstowego GNNa) zadanego typu.

b) Zbadaj jak głębokość grafowej sieci neuronowej wpływa na jakość uzyskiwanych reprezentacji (uwzględniając miarę AUC na zbiorze testowym). Wybierz kilka wartości dla liczby warstw i przedstaw wyniki w tabelce. Pamiętaj aby każdy eksperyment powtórzyć kilkukrotnie i podać wynik uśredniony wraz z odchyleniem standardowym. Skomentuj wyniki.

**[Zadanie dodatkowe]** *c) Znajdź w literaturze jak nazywa się to zaobserwowane zjawisko i wytłumacz własnymi słowami na czym polega.* 

In [ ]:
import pandas as pd
import torch


class NLayerGNN(nn.Module):
    def __init__(
        self,
        in_dim: int,
        hidden_dim: int,
        out_dim: int,
        layer_name: str,
        num_layers: int,
    ):
        super().__init__()
        
        # TU WPISZ KOD
        raise NotImplementedError()
    
    @staticmethod
    def _make_layer(layer_name: str, in_dim: int, out_dim: int):
        # TU WPISZ KOD
        raise NotImplementedError()
        
    def forward(self, x: torch.Tensor, edge_index: torch.Tensor) -> torch.Tensor:
        # TU WPISZ KOD
        raise NotImplementedError()
    
    
    
def evaluate_num_layers():
    # TU WPISZ KOD
    raise NotImplementedError()
    

evaluate_num_layers()